# Overview 

This run books automates the deployment of HDFS Cluster in arms raspberry hosts using docker. 

A key aspect of this example is that all HDFS nodes are in different hosts, and that requires some adjustments on the docker-compose files to allow all nodes to find the other nodes to allow the cluster to work as expected. This is done without using a external DNS service, just docker compose parameters. 


## Original Topology:

![HDFS Topology](./HDFS-Cluster-Topology.png "Toplogy")

The elements:

- 1 Master Node
- 2 Data Nodes
- 1 HDFS Client Apllication 


`/home/%(user)s/docker_host/hdfs` is the folder for the files of the images.

Based on repo: https://github.com/6za/hdfs-sample


## General Imports

In [1]:
import pandas as pd
import yaml
import os
import subprocess

## Define hosts roles

In [2]:
hosts = pd.read_csv("../common/hosts.csv")
supressed_columns = ['ip','user']


raspi_hosts = hosts[(hosts.hostname == "pi-node8") | (hosts.hostname == "pi-node7") | (hosts.hostname == "pi-node5") | (hosts.hostname == "pi-node4") ]
raspi_hosts['ip'].to_csv(r'raspi-ip.txt', header=False, index=None, sep=' ')


i386_hosts = hosts[(hosts.hostname == "nuc-03") ]
i386_hosts['ip'].to_csv(r'i386-ip.txt', header=False, index=None, sep=' ')


raspi_name_hosts = hosts[(hosts.hostname == "pi-node8")]
# Format name nodes - Destructive action
format_namenode = True

# Clean data nodes - Destructive action
clean_datanode = True

raspi_data_hosts = hosts[(hosts.hostname == "pi-node7") | (hosts.hostname == "pi-node5")]



raspi_test_hosts = hosts[(hosts.hostname == "pi-node4")]

In [3]:
i386_hosts.drop(columns=supressed_columns)

,hostname,arch,gpu
6,nuc-03,x86_64,0


In [4]:
raspi_hosts.drop(columns=supressed_columns)

,hostname,arch,gpu
7,pi-node4,armv7l,0
8,pi-node5,armv7l,0
10,pi-node7,armv7l,0
11,pi-node8,armv7l,0


In [5]:
raspi_name_hosts.drop(columns=supressed_columns)

,hostname,arch,gpu
11,pi-node8,armv7l,0


In [6]:
raspi_test_hosts.drop(columns=supressed_columns)

,hostname,arch,gpu
7,pi-node4,armv7l,0


In [7]:
raspi_data_hosts.drop(columns=supressed_columns)

,hostname,arch,gpu
8,pi-node5,armv7l,0
10,pi-node7,armv7l,0


## Clone Repo

In [8]:
%%bash
mkdir ~/repos
cd ~/repos
rm -rf ~/repos/hdfs-sample
git clone https://github.com/6za/hdfs-sample.git

mkdir: cannot create directory '/root/repos': File exists
Cloning into 'hdfs-sample'...


## Build Image on Nodes/Hosts

This step could be replaced by having a local/public image registry. For this example we build the image on each host by design of this example. 

In [9]:
!rm build_log.txt
for index, row in raspi_hosts.iterrows():
    docker_host = 'source /root/common/env.sh && export DOCKER_HOST=\"tcp://%(ip)s:2376\"' %  {"ip": row['ip']}
    command = "docker rmi local-hadoop:3.2.1 "
    result = subprocess.check_output("bash -c \"%s && %s  || : \" " %(docker_host,command)  , shell=True, encoding='utf-8')
    print("Image rm at "+ row['hostname'] + result )
    
for index, row in raspi_hosts.iterrows():
    docker_host = 'source /root/common/env.sh && export DOCKER_HOST=\"tcp://%(ip)s:2376\"' %  {"ip": row['ip']}
    command = "docker build --file ~/repos/hdfs-sample/Dockerfile.armhf ~/repos/hdfs-sample/ -t local-hadoop:3.2.1 >> build_log.txt"
    result = subprocess.check_output("bash -c \"%s && %s  || : \" " %(docker_host,command)  , shell=True, encoding='utf-8')
    command = "docker images | grep  local-hadoop "
    images_list = subprocess.check_output("bash -c \"%s && %s  || : \" " %(docker_host,command)  , shell=True, encoding='utf-8')
    print("Image Built at "+ row['hostname'] + images_list)

Image rm at pi-node4
Image rm at pi-node5
Image rm at pi-node7
Image rm at pi-node8
Image Built at pi-node4local-hadoop         3.2.1               ca6589c3b5de        22 hours ago        1.35GB

Image Built at pi-node5local-hadoop         3.2.1               5c6a8d08da69        22 hours ago        1.35GB

Image Built at pi-node7local-hadoop         3.2.1               c7d7062b8fd9        22 hours ago        1.35GB

Image Built at pi-node8local-hadoop         3.2.1               5a08de6c74e7        22 hours ago        1.35GB



- Update datanode compose with with namenode address.

## Generate SSH Key

Main Command: `ssh-keygen -q -t rsa -N '' -f id_rsa`

> An `i386` image/host is used for compatibility and speed to generate the certs. 

In [10]:
for index, row in i386_hosts.iterrows():
    command_mk = "ssh %(user)s@%(ip)s mkdir -p /var/tmp/certs" % {"user":row['user'], "ip": row['ip']}
    result_mk = subprocess.check_output("bash -c \"%s  2>&1 || : \" " %(command_mk)  , shell=True, encoding='utf-8')
    command = "docker pull 6zar/ssh_tool && docker run -v /var/tmp/certs:/root 6zar/ssh_tool | grep identification"
    result = subprocess.check_output("bash -c \"%s && %s  2>&1 || : \" " %(docker_host,command)  , shell=True, encoding='utf-8')
    command_cp = "scp %(user)s@%(ip)s:/var/tmp/certs/* ." % {"user":row['user'], "ip": row['ip']}
    result_cp = subprocess.check_output("bash -c \"%s  2>&1 || : \" " %(command_cp)  , shell=True, encoding='utf-8')
    !cat ./id_rsa.pub > ./authorized_keys
    command_rm = "ssh %(user)s@%(ip)s rm -rf /var/tmp/certs" % {"user":row['user'], "ip": row['ip']}
    result_rm = subprocess.check_output("bash -c \"%s  2>&1 || : \" " %(command_rm)  , shell=True, encoding='utf-8')



## Deploy Keys on all nodes

Accordingly with some HDFS stackoverflow this is required to allow nodes to exchange files.

This will copy `id_rsa*` files and `authorized_keys` keys to all hosts.


In [11]:
for index, row in raspi_hosts.iterrows():
    command_rmdir = "ssh %(user)s@%(ip)s rm -rf /home/%(user)s/docker_host/hdfs" % {"user":row['user'], "ip": row['ip']}
    result_rmdir = subprocess.check_output("bash -c \"%s  2>&1 || : \" " %(command_rmdir)  , shell=True, encoding='utf-8')
    command_mkdir = "ssh %(user)s@%(ip)s mkdir -p /home/%(user)s/docker_host/hdfs" % {"user":row['user'], "ip": row['ip']}
    result_mkdir = subprocess.check_output("bash -c \"%s  2>&1 || : \" " %(command_mkdir)  , shell=True, encoding='utf-8')
    # Copy idrsa
    command_cp = "scp ./id_rsa*  %(user)s@%(ip)s:/home/%(user)s/docker_host/hdfs/" % {"user":row['user'], "ip": row['ip']}
    result_cp = subprocess.check_output("bash -c \"%s  2>&1 || : \" " %(command_cp)  , shell=True, encoding='utf-8')
    command_cp = "scp ./authorized*  %(user)s@%(ip)s:/home/%(user)s/docker_host/hdfs/" % {"user":row['user'], "ip": row['ip']}
    result_cp = subprocess.check_output("bash -c \"%s  2>&1 || : \" " %(command_cp)  , shell=True, encoding='utf-8')
    print("Keys deployed at "+ row['hostname'])


Keys deployed at pi-node4
Keys deployed at pi-node5
Keys deployed at pi-node7
Keys deployed at pi-node8


## Copy HDFS configs to all nodes

In [12]:
for index, row in raspi_hosts.iterrows():
    command_rmdir = "ssh %(user)s@%(ip)s rm -rf /home/%(user)s/docker_host/hdfs/conf" % {"user":row['user'], "ip": row['ip']}
    result_rmdir = subprocess.check_output("bash -c \"%s  2>&1 || : \" " %(command_rmdir)  , shell=True, encoding='utf-8')
    command_mkdir = "ssh %(user)s@%(ip)s mkdir -p /home/%(user)s/docker_host/hdfs/conf" % {"user":row['user'], "ip": row['ip']}
    result_mkdir = subprocess.check_output("bash -c \"%s  2>&1 || : \" " %(command_mkdir)  , shell=True, encoding='utf-8')
    command_cp = "scp ~/repos/hdfs-sample/hdfs-conf/*  %(user)s@%(ip)s:/home/%(user)s/docker_host/hdfs/conf/" % {"user":row['user'], "ip": row['ip']}
    result_cp = subprocess.check_output("bash -c \"%s  2>&1 || : \" " %(command_cp)  , shell=True, encoding='utf-8')
    command = "ssh %(user)s@%(ip)s ls /home/%(user)s/docker_host/hdfs/conf" % {"user":row['user'], "ip": row['ip']}
    result = subprocess.check_output("bash -c \"%s  2>&1 || : \" " %(command)  , shell=True, encoding='utf-8')
    print("Configs deployed at "+ row['hostname'])


Configs deployed at pi-node4
Configs deployed at pi-node5
Configs deployed at pi-node7
Configs deployed at pi-node8


## Namenode Deploy

In [13]:
for index, row in raspi_name_hosts.iterrows():
    docker_host = 'source /root/common/env.sh && export DOCKER_HOST=\"tcp://%(ip)s:2376\"' %  {"ip": row['ip']}
    command = "docker-compose -f ~/repos/hdfs-sample/docker-compose-namenode.yaml down"  
    result = subprocess.check_output("bash -c \"%s && %s  || : \" " %(docker_host,command)  , shell=True, encoding='utf-8')        
    print("HDFS Namenode Stopped")

HDFS Namenode Stopped


In [14]:
# Create shared folder
# Create Format disk
# This is a destructive action(old data will be removed)
if format_namenode:    
    for index, row in raspi_name_hosts.iterrows():
        docker_host = 'source /root/common/env.sh && export DOCKER_HOST=\"tcp://%(ip)s:2376\"' %  {"ip": row['ip']}
        command_rmdir = "ssh %(user)s@%(ip)s sudo rm -rf /home/%(user)s/docker_host/persisted/hdfs" % {"user":row['user'], "ip": row['ip']}
        result_rmdir = subprocess.check_output("bash -c \"%s  2>&1 || : \" " %(command_rmdir)  , shell=True, encoding='utf-8')    
        command_lsdir = "ssh %(user)s@%(ip)s ls -la /home/%(user)s/docker_host/persisted/" % {"user":row['user'], "ip": row['ip']}
        result_lsdir = subprocess.check_output("bash -c \"%s  2>&1 || : \" " %(command_lsdir)  , shell=True, encoding='utf-8')    
        print(result_lsdir)
        command_mkdir = "ssh %(user)s@%(ip)s mkdir -p /home/%(user)s/docker_host/persisted/hdfs" % {"user":row['user'], "ip": row['ip']}
        result_mkdir = subprocess.check_output("bash -c \"%s  2>&1 || : \" " %(command_mkdir)  , shell=True, encoding='utf-8')    
        command = "docker run  -v /home/pi/docker_host/hdfs/id_rsa:/root/.ssh/id_rsa -v /home/pi/docker_host/hdfs/id_rsa.pub:/root/.ssh/id_rsa.pub  -v /home/pi/docker_host/hdfs/conf:/opt/hadoop/etc/hadoop -v /home/pi/docker_host/persisted/hdfs:/data/dfs/name local-hadoop:3.2.1 /opt/hadoop/bin/hdfs namenode -format" % {"user":row['user'], "ip": row['ip']}   
        result = subprocess.check_output("bash -c \"%s && %s  || : \" " %(docker_host,command)  , shell=True, encoding='utf-8')        
        print("HDFS Namenode Formated")

total 8
drwxr-xr-x 2 pi pi 4096 Jan 28 20:26 .
drwxr-xr-x 4 pi pi 4096 Jan 28 20:26 ..

HDFS Namenode Formated


In [15]:
#Start name node
for index, row in raspi_name_hosts.iterrows():
    docker_host = 'source /root/common/env.sh && export DOCKER_HOST=\"tcp://%(ip)s:2376\"' %  {"ip": row['ip']}
    command = "docker-compose -f ~/repos/hdfs-sample/docker-compose-namenode.yaml up -d"  
    result = subprocess.check_output("bash -c \"%s && %s  || : \" " %(docker_host,command)  , shell=True, encoding='utf-8')        
    print("HDFS Namenode Started")

HDFS Namenode Started


## Datanodes Deploy

In [16]:
namenode_ip = raspi_name_hosts.iloc[0]['ip']
namenode_config = 'hadoop-namenode:%(namenode_ip)s' %  {"namenode_ip": namenode_ip}
hosts_list = [namenode_config]

for index, row in raspi_data_hosts.iterrows():
    hostname='hdfs-data-' + row['hostname']
    datanode_config='%(hostname)s:%(ip)s' %  {"hostname": hostname,"ip": row['ip'] }
    hosts_list.append(datanode_config)

node_count = 0
with open('/root/repos/hdfs-sample/docker-compose-datanode.yaml',"r") as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    config = yaml.load(file, Loader=yaml.FullLoader)
    namenode_config = 'hadoop-namenode:%(namenode_ip)s' %  {"namenode_ip": namenode_ip}
    docker_host_config = hosts_list
    config['services']['hadoop-datanode']['extra_hosts'] = docker_host_config
    for index, row in raspi_data_hosts.iterrows():
        node_count = node_count + 1
        config['services']['hadoop-datanode']['hostname'] = 'hdfs-data-' + row['hostname']
        with open(r'docker-compose-datanode-current.yaml', 'w') as outputfile:
            documents = yaml.dump(config, outputfile)    
        print('\x1b[1;35m'+ row['hostname']+'\x1b[0m')
        docker_host = 'source /root/common/env.sh && export DOCKER_HOST=\"tcp://%(ip)s:2376\"' %  {"ip": row['ip']}
        command = "docker-compose -f docker-compose-datanode-current.yaml down"
        result = subprocess.check_output("bash -c \"%s && %s  || : \" " %(docker_host,command)  , shell=True, encoding='utf-8')        
        if clean_datanode: 
            command_rmdir = "ssh %(user)s@%(ip)s sudo rm -rf /home/%(user)s/docker_host/persisted/hdfs" % {"user":row['user'], "ip": row['ip']}
            result_rmdir = subprocess.check_output("bash -c \"%s  2>&1 || : \" " %(command_rmdir)  , shell=True, encoding='utf-8')    
            command_mkdir = "ssh %(user)s@%(ip)s mkdir -p /home/%(user)s/docker_host/persisted/hdfs" % {"user":row['user'], "ip": row['ip']}
            result_mkdir = subprocess.check_output("bash -c \"%s  2>&1 || : \" " %(command_mkdir)  , shell=True, encoding='utf-8')
            print("  Datanode data clean:" + row['hostname'])
        command = "docker-compose -f docker-compose-datanode-current.yaml up -d"  
        result = subprocess.check_output("bash -c \"%s && %s  || : \" " %(docker_host,command)  , shell=True, encoding='utf-8')        
        print("  Datanode started:" + row['hostname'])




pi-node5
  Datanode data clean:pi-node5
  Datanode started:pi-node5
pi-node7
  Datanode data clean:pi-node7
  Datanode started:pi-node7


## Check Cluster post deployment

In [17]:
for index, row in raspi_hosts.iterrows():
    docker_host = 'source /root/common/env.sh && export DOCKER_HOST=\"tcp://%(ip)s:2376\"' %  {"ip": row['ip']}
    command = "docker ps"    
    result = subprocess.check_output("bash -c \"%s && %s  || : \" " %(docker_host,command)  , shell=True, encoding='utf-8')
    print('\x1b[1;35m'+ row['hostname']+'\x1b[0m')
    print(result)


pi-node4
CONTAINER ID        IMAGE                        COMMAND                  CREATED             STATUS              PORTS                    NAMES
5dc6b94328d9        prom/node-exporter:v0.18.0   "/bin/node_exporter …"   2 weeks ago         Up 2 days           0.0.0.0:9100->9100/tcp   nodeexporter

pi-node5
CONTAINER ID        IMAGE                        COMMAND                  CREATED             STATUS              PORTS                                                                                                                                                                                                                NAMES
6f503d25c2f6        local-hadoop:3.2.1           "/opt/hadoop/bin/hdf…"   14 seconds ago      Up 11 seconds       22/tcp, 8020/tcp, 8088/tcp, 0.0.0.0:50010->50010/tcp, 9000/tcp, 0.0.0.0:50020->50020/tcp, 50030/tcp, 0.0.0.0:50060->50060/tcp, 50070/tcp, 50090/tcp, 0.0.0.0:50075->50075/tcp, 50470/tcp, 0.0.0.0:50475->50475/tcp   09-hdfs-on-arm-pi_hadoop

## Run test application to validate cluster

In [18]:
add_host_list=" "    
for host in hosts_list: 
    add_host_list = add_host_list +" --add-host=" +host + "  "
    
    
for index, row in raspi_test_hosts.iterrows():
    print("Add file to HDFS:")
    docker_host = 'source /root/common/env.sh && export DOCKER_HOST=\"tcp://%(ip)s:2376\"' %  {"ip": row['ip']}
    command = "docker run  %(hostlist)s local-hadoop:3.2.1 /opt/hadoop/bin/hadoop fs -put /opt/hadoop/etc/hadoop/hdfs-site.xml  hdfs://hadoop-namenode:9000/ " % {"hostlist": add_host_list}
    result = subprocess.check_output("bash -c \"%s && %s || :   \" " %(docker_host,command)  , shell=True, encoding='utf-8')
    print(result)
    print("Check file on HDFS:")
    command = "docker run %(hostlist)s  local-hadoop:3.2.1 /opt/hadoop/bin/hadoop fs -ls  hdfs://hadoop-namenode:9000/ " % {"hostlist": add_host_list}
    result = subprocess.check_output("bash -c \"%s && %s || :  \" " %(docker_host,command)  , shell=True, encoding='utf-8')
    print(result)
    print("Open File to HDFS:")
    command = "docker run  %(hostlist)s local-hadoop:3.2.1 /opt/hadoop/bin/hadoop fs -cat hdfs://hadoop-namenode:9000/hdfs-site.xml" % {"hostlist": add_host_list}
    result = subprocess.check_output("bash -c \"%s && %s || :   \" " %(docker_host,command)  , shell=True, encoding='utf-8')
    print(result)




Add file to HDFS:

Check file on HDFS:
Found 1 items
-rw-r--r--   3 root supergroup        775 2020-01-29 01:27 hdfs://hadoop-namenode:9000/hdfs-site.xml

Open File to HDFS:
<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet type="text/xsl" href="configuration.xsl"?>
<!--
  Licensed under the Apache License, Version 2.0 (the "License");
  you may not use this file except in compliance with the License.
  You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

  Unless required by applicable law or agreed to in writing, software
  distributed under the License is distributed on an "AS IS" BASIS,
  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  See the License for the specific language governing permissions and
  limitations under the License. See accompanying LICENSE file.
-->

<!-- Put site-specific property overrides in this file. -->

<configuration>

</configuration>

